In [17]:
import pandas as pd
import numpy as np

In [13]:
product_types = """
Брюки для детей школьного возраста из джинсовой ткани, шт.
Пеленки для новорожденных, шт.
Костюм спортивный для детей школьного возраста, шт.
Майка, футболка мужская бельевая, шт.
Футболка детская, шт.
Колготки женские эластичные, шт.
Носки мужские, пара
Кроссовые туфли для детей, пара
Кроссовые туфли для взрослых, пара
Мыло хозяйственное, 200 г
Порошок стиральный, кг
Мыло туалетное, 100 г
Шампунь, 250 мл
Паста зубная, 100 г (100 мл)
Щетка зубная, шт.
Сигареты с фильтром, пачка
Спички, коробок
Электропылесос напольный, шт.
Бумага туалетная, рулон
Подгузники детские бумажные, 10 шт.
Прокладки женские гигиенические, 10 шт.
Телевизор, шт.
Смартфон, шт.
Плиты древесностружечные, ориентированно-стружечные, м2
Доска обрезная, м3
Легковой автомобиль отечественный новый, шт.
Легковой автомобиль иностранной марки новый, шт.
Бензин автомобильный
Бензин автомобильный марки АИ-92, л
Бензин автомобильный марки АИ-95, л
Дизельное топливо, л
Бензин автомобильный марки АИ-98 и выше, л
Корвалол, 25 мл
Валидол, 60 мг, 10 таблеток
Метамизол натрия (Анальгин отечественный), 500 мг, 10 таблеток
Комбинированные анальгетики, 10 таблеток
Нимесулид, 100 мг, 10 таблеток
Левомеколь, мазь, 40 г
Аллохол, 50 таблеток
Ренгалин, 10 таблеток
Эргоферон, 20 таблеток
Поливитамины, 10 шт.
Сухие корма для домашних животных, кг
""".split("\n")

product_types = [item for item in product_types if len(item) > 0]
# product_types = {item: "" for item in product_types}
data = {
	"product_types": product_types,
	"category": len(product_types) * ['']
}
pd.DataFrame(data=data).to_excel("types.xlsx",index=False)

In [ ]:
product_types = pd.read_excel("./data/types.xlsx")
product_types_dict = {product_types['product_types'].iloc[i]:product_types['category'].iloc[i].strip() for i in range(len(product_types['product_types'])) }

product_types_dict

In [44]:
product_types_set = list(set(list(map(str.strip, list(product_types['category'].values)))))

In [18]:
def row_stringify(row):
	row = map(str, list(row.values))
	str_row = "".join(row)
	return str_row

def coords(dataset, name):
	row_index = 0
	for i in range(len(dataset)):
		row = dataset.iloc[i]
		row_str = row_stringify(row)
		if name in row_str:
			row_index = i
			break
	col_index = 0
	for j in range(len(dataset.columns)):
		col = dataset.iloc[row_index, j]
		if name in str(col):
			col_index = j
			break
	
	return np.array([row_index, col_index], dtype=np.int64)

In [19]:
dataset = pd.read_excel("./data/Средние_потребительские_цены_на_непродовольственные_товарыСредние.xlsx")

d:\programming\AI\ai_env\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [35]:
area_coords = []

for i in range(len(dataset.columns)):
	item = dataset.iloc[2, i]
	item = str(item)
	if len(item) > 5:
		if len(area_coords) > 0:
			area_coords[-1].append([2, i])
		area_coords.append(
			[
				item,
				[2, i]
			]
		)
area_coords[-1].append([2, len(dataset.columns)])

area_coords.insert(0,
	['', [2, 1], [2, area_coords[0][1][-1]]]
)

In [36]:
area_coords

[['', [2, 1], [2, 5]],
 ['Центральный федеральный округ', [2, 5], [2, 81]],
 ['Северо-Западный федеральный округ', [2, 81], [2, 129]],
 ['Южный федеральный округ', [2, 129], [2, 165]],
 ['Северо-Кавказский федеральный округ', [2, 165], [2, 197]],
 ['Приволжский федеральный округ', [2, 197], [2, 257]],
 ['Уральский федеральный округ', [2, 257], [2, 285]],
 ['Сибирский федеральный округ', [2, 285], [2, 329]],
 ['Дальневосточный федеральный округ', [2, 329], [2, 377]]]

In [80]:
dataset.iloc[2, 376]

nan

In [82]:
new_dataset_columns = [
	"Дата",
	"Страна",
	"Наименование округа",
	"Наименование субъекта",
	"Тип агрегации",
	"Категория товара или услуги",
	"Наименование товара",
	"Средняя цена",
	"Позиция в рейтинге"
]

flat_dataset = {name: [] for name in new_dataset_columns}

def my_argsort(my_list, add=1):
	sorted_list = sorted(my_list)
	return [sorted_list.index(item)+add for item in my_list]

area_name = ""
for i in range(len(area_coords)):
	start_coords = area_coords[i]
	start_y, start_x = start_coords[1]
	end_y, end_x = start_coords[2]

	subject_name = dataset.iloc[start_y, start_x]
	for col in range(start_x, end_x):
		area_prices = {prod_type: [] for prod_type in product_types_set} 

		for j in range(7, len(dataset)-1):
			product_name = dataset.iloc[j, 0].strip()
			
			if len(str(dataset.iloc[3, col])) > 0 and not dataset.iloc[3, col] is np.nan:
				area_name = dataset.iloc[3, col]
			
			avg_price = dataset.iloc[j, col]
			date = dataset.iloc[6, col]
			products_type = product_types_dict[product_name]
			flat_dataset["Дата"].append(date)
			flat_dataset["Страна"].append("Российская Федерация")
			flat_dataset["Наименование округа"].append(subject_name)
			flat_dataset["Наименование субъекта"].append(area_name)
			flat_dataset["Тип агрегации"].append("")
			flat_dataset["Категория товара или услуги"].append(products_type)
			flat_dataset["Наименование товара"].append(product_name)
			flat_dataset["Средняя цена"].append(avg_price)
			flat_dataset["Позиция в рейтинге"].append("")
			# area_prices.append(avg_price)
			area_prices[products_type].append([len(flat_dataset["Позиция в рейтинге"]) - 1, avg_price])
		# print(area_prices)
		# area_prices.sort()
		# for k in range(len(area_prices)):
		# 	avg_price = flat_dataset["Средняя цена"][-(k+1)]
		# 	position = area_prices.index(avg_price) + 1
		# 	flat_dataset["Позиция в рейтинге"][-(k+1)] = position

		for product_type in product_types_set:
			prices_list = area_prices[product_type]
			prices_list_avg = [item[1] for item in prices_list]
			# print(prices_list_avg)
			prices_list_avg = my_argsort(prices_list_avg)
			# print(prices_list_avg)
			# break
			for k in range(len(prices_list)):
				pos = prices_list[k][0]
				position = prices_list_avg[k]
				flat_dataset["Позиция в рейтинге"][pos] = position
		# break


In [83]:
pd.DataFrame(data=flat_dataset).to_excel("./test.xlsx", index=False)

In [48]:
import pprint
pprint.pprint({'Гигиена': [[16, 53.91], [17, 209.87], [18, 57.2], [19, 203.94], [20, 147.13], [21, 122.01], [25, 22.19], [27, 137.01]], 'Детские товары': [[8, 225.23], [26, 232.58]], 'Товары для дома': [[23, 2.72], [30, 414.41], [31, 20046.89], [49, 291.36]], 'Медикаменты': [[39, 36.22], [40, 47.82], [41, 26.15], [42, 136.96], [43, 109.21], [44, 188.41], [45, 80.97], [46, 130.22], [47, 474], [48, 76.96]], 'Алкогольная и табачная продукция': [[22, 155.73]], 'Обувь': [[14, 1719.69], [15, 3475.3]], 'Транспорт': [[32, 955754.31], [33, 1914497.34]], 'Одежда': [[7, 1617.62], [9, 2173], [10, 443.06], [11, 462.89], [12, 261.3], [13, 111.01]], 'Автотовары': [[34, 50.57], [35, 47.17], [36, 51.23], [37, 54.42], [38, 60.12]], 'Электроника': [[24, 9276.7], [28, 34661.78], [29, 12886.98]]})

{'Автотовары': [[34, 50.57],
                [35, 47.17],
                [36, 51.23],
                [37, 54.42],
                [38, 60.12]],
 'Алкогольная и табачная продукция': [[22, 155.73]],
 'Гигиена': [[16, 53.91],
             [17, 209.87],
             [18, 57.2],
             [19, 203.94],
             [20, 147.13],
             [21, 122.01],
             [25, 22.19],
             [27, 137.01]],
 'Детские товары': [[8, 225.23], [26, 232.58]],
 'Медикаменты': [[39, 36.22],
                 [40, 47.82],
                 [41, 26.15],
                 [42, 136.96],
                 [43, 109.21],
                 [44, 188.41],
                 [45, 80.97],
                 [46, 130.22],
                 [47, 474],
                 [48, 76.96]],
 'Обувь': [[14, 1719.69], [15, 3475.3]],
 'Одежда': [[7, 1617.62],
            [9, 2173],
            [10, 443.06],
            [11, 462.89],
            [12, 261.3],
            [13, 111.01]],
 'Товары для дома': [[23, 2.72], [30, 414.4

In [49]:
prices = {'Автотовары': [[34, 50.57],
                [35, 47.17],
                [36, 51.23],
                [37, 54.42],
                [38, 60.12]],
 'Алкогольная и табачная продукция': [[22, 155.73]],
 'Гигиена': [[16, 53.91],
             [17, 209.87],
             [18, 57.2],
             [19, 203.94],
             [20, 147.13],
             [21, 122.01],
             [25, 22.19],
             [27, 137.01]],
 'Детские товары': [[8, 225.23], [26, 232.58]],
 'Медикаменты': [[39, 36.22],
                 [40, 47.82],
                 [41, 26.15],
                 [42, 136.96],
                 [43, 109.21],
                 [44, 188.41],
                 [45, 80.97],
                 [46, 130.22],
                 [47, 474],
                 [48, 76.96]],
 'Обувь': [[14, 1719.69], [15, 3475.3]],
 'Одежда': [[7, 1617.62],
            [9, 2173],
            [10, 443.06],
            [11, 462.89],
            [12, 261.3],
            [13, 111.01]],
 'Товары для дома': [[23, 2.72], [30, 414.41], [31, 20046.89], [49, 291.36]],
 'Транспорт': [[32, 955754.31], [33, 1914497.34]],
 'Электроника': [[24, 9276.7], [28, 34661.78], [29, 12886.98]]}



In [50]:
some = [[34, 50.57],
		[35, 47.17],
		[36, 51.23],
		[37, 54.42],
		[38, 60.12]]



In [66]:
def my_argsort(my_list):
	sorted_list = sorted(my_list)
	return [sorted_list.index(item) for item in my_list]
	
myList = [
36.22,
47.82,
26.15,
136.9,
109.2,
188.4,
80.97,
130.2,
474,
76.96,
]

my_argsort(myList)


[1, 2, 0, 7, 5, 8, 4, 6, 9, 3]

In [64]:
sorted(myList)

[26.15, 36.22, 47.82, 76.96, 80.97, 109.2, 130.2, 136.9, 188.4, 474]